In this experiment we gonna try to initialize Transformer model with good embeddings, learnt from monolingual corpora.

In [2]:
import sys; sys.path.append('..')

import os
import json
import argparse
from random import choice
from time import time
from datetime import timedelta

import numpy as np; np.random.seed(42)
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from pandas import ewma
from IPython.display import clear_output

from vocab import Vocab
from src.training_utils import *
from lib.tensor_utils import infer_mask, initialize_uninitialized_variables, all_shapes_equal
from lib.utils import save_score

from models.transformer_other import Model as Transformer

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
import os
from os import path

config = {
    'data_path': '../data/generated',
    'hp_file_path': '../hp_files/trans_default.json',
    'use_early_stopping': True,
    'early_stopping_last_n': 10,
    'max_epochs': 1000,
    'max_time_seconds': 600,
    'batch_size_for_inference': 16,
    'max_len': 200,
    'validate_every_num_epochs': 5,
    'warm_up_num_epochs': 10
}

model_name = 'transformer'
model_path = path.join('../trained_models', model_name)
if not os.path.isdir(model_path): os.mkdir(model_path)

src_train_path = path.join(config.get('data_path'), 'src_train.tok.bpe')
trg_train_path = path.join(config.get('data_path'), 'trg_train.tok.bpe')
src_val_path = path.join(config.get('data_path'), 'src_val.tok.bpe')
trg_val_path = path.join(config.get('data_path'), 'trg_val.tok.bpe')

src_train = open(src_train_path, 'r', encoding='utf-8').read().splitlines()[:1000]
trg_train = open(trg_train_path, 'r', encoding='utf-8').read().splitlines()[:1000]
src_val = open(src_val_path, 'r', encoding='utf-8').read().splitlines()[:100]
trg_val = open(trg_val_path, 'r', encoding='utf-8').read().splitlines()[:100]

vocab_src = Vocab.from_file(path.join(config.get('data_path'), 'vocab.src'))
vocab_trg = Vocab.from_file(path.join(config.get('data_path'), 'vocab.trg'))
max_len = config.get('max_len', 200)

# Hyperparameters
hp = json.load(open(config.get('hp_file_path'), 'r', encoding='utf-8')) if config.get('hp_file_path') else {}

In [4]:
sess = tf.InteractiveSession()
# optimizer = create_optimizer(hp)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

inp = tf.placeholder(tf.int32, [None, None])
out = tf.placeholder(tf.int32, [None, None])

In [5]:
#############################################
### Initializing our main model
#############################################
model = Transformer(model_name, vocab_src, vocab_trg, **hp)

logprobs = model.symbolic_score(inp, out, is_train=True)[:,:tf.shape(out)[1]]
nll = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logprobs, labels=out)
loss = nll * infer_mask(out, vocab_trg.eos, dtype=tf.float32)
loss = tf.reduce_sum(loss, axis=1)
loss = tf.reduce_mean(loss)
weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, model_name)

# grads = tf.gradients(loss, weights)
# grads = tf.clip_by_global_norm(weights, 100)[0]

# train_step = optimizer.apply_gradients(zip(grads, weights))
train_step = optimizer.minimize(loss, var_list=weights)
#############################################

In [6]:
batch_size = hp.get('batch_size', 32)
epoch = 0
training_start_time = time()
loss_history = []
val_scores = []

num_iters_done = 0
should_start_next_epoch = True # We need this var to break outer loop

initialize_uninitialized_variables(sess)

ResourceExhaustedError: OOM when allocating tensor of shape [512,2048] and type float
	 [[Node: transformer/dec_ffn-2/conv1/W/Adam/Initializer/zeros = Const[_class=["loc:@transformer/dec_ffn-2/conv1/W"], dtype=DT_FLOAT, value=Tensor<type: float shape: [512,2048] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'transformer/dec_ffn-2/conv1/W/Adam/Initializer/zeros', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/universome/pyvenvs/zoo/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/universome/pyvenvs/zoo/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/universome/pyvenvs/zoo/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in st

In [ ]:
def save_model():
    save_path = '{}/model.npz'.format(model_path)
    print('Saving the model into %s' %save_path)

    w_values = sess.run(weights)
    weights_dict = {w.name: w_val for w, w_val in zip(weights, w_values)}
    np.savez(save_path, **weights_dict)

def validate(val_scores):
    """
    Returns should_continue flag, which tells us if we should continue or early stop
    """
    should_continue = True

    # if config.get('warm_up_num_epochs') and config.get('warm_up_num_epochs') > epoch:
    #     print('Skipping validation, becaused is not warmed up yet')
    #     return should_continue

    print('Validating')
    val_score = compute_bleu_for_model(model, sess, vocab_src, vocab_trg, src_val,
                                       trg_val, model_name, config, max_len=max_len)
    val_scores.append(val_score)
    print('Validation BLEU: {:0.3f}'.format(val_score))

    # Save model if this is our best model
    if np.argmax(val_scores) == len(val_scores)-1:
        print('Saving model because it has the highest validation BLEU.')
        save_model()

    if config.get('use_early_stopping') and should_stop_early(val_scores, config.get('early_stopping_last_n')):
        print('Model did not improve for last %s steps. Early stopping.' % config.get('early_stopping_last_n'))
        should_continue = False

    if config.get('warm_up_num_epochs') > epoch:
        return True
    else:
        return should_continue

In [ ]:
while should_start_next_epoch:
    batches = batch_generator_over_dataset(src_train, trg_train, batch_size, batches_per_epoch=None)
    
    for batch_src, batch_trg in tqdm(batches):
        # Note: we don't use vocab.tokenize_many(batch, max_len=max_len)
        # cuz it forces batch length to be that long and we often get away with much less
        batch_src_ix = vocab_src.tokenize_many(batch_src)[:, :max_len]
        batch_trg_ix = vocab_trg.tokenize_many(batch_trg)[:, :max_len]

        feed_dict = {inp: batch_src_ix, out: batch_trg_ix}

        loss_t = sess.run([train_step, loss], feed_dict)[1]
        loss_history.append(np.mean(loss_t))

        if num_iters_done % 10 == 0:
            clear_output(True)
            plt.figure(figsize=[15,8])
            plt.title('Batch loss')
            plt.plot(loss_history)
            plt.plot(ewma(np.array(loss_history), span=100))
            plt.grid()
            plt.show()

        num_iters_done += 1

        if config.get('max_time_seconds'):
            seconds_elapsed = time()-training_start_time

            if seconds_elapsed > config.get('max_time_seconds'):
                print('Maximum allowed training time reached. Training took %s. Stopping.' % seconds_elapsed)
                should_start_next_epoch = False
                break

    if epoch % config.get('validate_every_num_epochs') == 0 and should_start_next_epoch:
        should_start_next_epoch = validate(val_scores)

    if config.get('max_epochs') and config.get('max_epochs') == epoch:
        print('Maximum amount of epochs reached. Stopping.')
        should_start_next_epoch = False
        break

    epoch +=1